In [3]:
import numpy as np
import joblib
from tensorflow.keras.models import load_model

# Load the trained RandomForest model
model_rf = joblib.load("G:/mini project/multiclass_classifier.pkl")

# Load the trained RNN model
model_rnn = load_model("G:/mini project/best_rnn_model.keras")

# Function to predict alarms using RF model
def predict_alarms(hr, spo2, etco2, awrr):
    # Prepare input for RF model
    input_data = np.array([[hr, spo2, etco2, awrr]])
    
    # Predict alarms
    alarms = model_rf.predict(input_data)
    return alarms[0]  # Return the binary predictions

# Function to prepare input for RNN
def prepare_rnn_input(hr_values, spo2_values, etco2_values, awrr_values, alarms):
    # The error occurs because 'alarms' is a single prediction (shape of 4)
    # while the other inputs are arrays of length 20
    
    # Create an array of alarms for each time step
    # We'll use the same alarm values for all time steps
    alarm_array = np.tile(alarms, (len(hr_values), 1))
    
    # Combine sensor data and alarms
    combined_data = np.column_stack((hr_values, spo2_values, etco2_values, awrr_values, alarm_array))
    
    # Reshape for RNN input (n_samples, time_steps, n_features)
    rnn_input = combined_data.reshape(1, combined_data.shape[0], combined_data.shape[1])
    return rnn_input

# Function to predict with RNN
def predict_rnn(rnn_input):
    # Predict with RNN
    predictions = model_rnn.predict(rnn_input)
    return predictions[0]  # Return the predicted inSEV and inO2

# Function to predict alarms and RNN output
def predict_with_rf_and_rnn(hr_values, spo2_values, etco2_values, awrr_values):
    # Predict alarms with RF model
    alarms = predict_alarms(hr_values[-1], spo2_values[-1], etco2_values[-1], awrr_values[-1])
    
    # Prepare input for RNN
    rnn_input = prepare_rnn_input(hr_values[-20:], spo2_values[-20:], etco2_values[-20:], awrr_values[-20:], alarms)
    
    # Predict with RNN
    rnn_predictions = predict_rnn(rnn_input)
    
    return alarms, rnn_predictions

# Example sensor data (replace with actual data)
hr_values = [80, 82, 81, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63]
spo2_values = [98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79]
etco2_values = [40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21]
awrr_values = [16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3]

# Predict alarms and RNN output
alarms, rnn_predictions = predict_with_rf_and_rnn(hr_values, spo2_values, etco2_values, awrr_values)

# Print results
print("Predicted Alarms:", alarms)
print("RNN Predictions (inSEV, inO2):", rnn_predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 445ms/step
Predicted Alarms: [0 0 0 0]
RNN Predictions (inSEV, inO2): [0.52044034 0.28123534]
